In [27]:
import os
import sys
from collections import OrderedDict
sys.path.append("../..")
import spatialpy
spatialpy.__file__
import matplotlib.pyplot as plt
import numpy
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
class All(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] < 0.0 or x[0] > 1.0 or \
           x[1] < 0.0 or x[1] > 1.0:
            return True
        return False

In [29]:
class turingPattern(spatialpy.Model):

   def __init__(self, model_name="turingPattern"):
        spatialpy.Model.__init__(self, model_name)

        diff_const = 0.1
   
        A = spatialpy.Species(name="A", diffusion_constant=diff_const)
        B = spatialpy.Species(name="B", diffusion_constant=diff_const)
        D = spatialpy.Species(name="D", diffusion_constant=diff_const)
        E = spatialpy.Species(name="E", diffusion_constant=diff_const)
        X = spatialpy.Species(name="X", diffusion_constant=diff_const)
        Y = spatialpy.Species(name="Y", diffusion_constant=diff_const)
        self.add_species([A,B,D,E,X,Y])

        self.mesh = spatialpy.Mesh.create_2D_domain(nx=10, ny=10, xlim=[-1, 1], ylim=[-1, 1], fixed=True)

        self.set_type(All(), 1)

        Rate1 = spatialpy.Parameter("Rate1", expression=5000.0)
        Rate2 = spatialpy.Parameter("Rate2", expression=50.0)
        Rate3 = spatialpy.Parameter("Rate3", expression=0.0005)
        Rate4 = spatialpy.Parameter("Rate4", expression=5.0)
        self.add_parameter([Rate1, Rate2, Rate3, Rate4])

        Reaction1 = spatialpy.Reaction(reactants={A:1}, products={A:1, X:1}, rate=Rate1)
        Reaction2 = spatialpy.Reaction(reactants={B:1, X:1}, products={Y:1, D:1}, propensity_function="Rate2 * B * X")
        Reaction3 = spatialpy.Reaction(reactants={X:2, Y:1}, products={X:3}, propensity_function="(Rate3 * X * (X-1))/2")
        Reaction4 = spatialpy.Reaction(reactants={X:1}, products={E:1}, propensity_function="Rate4 * X")
        self.add_reaction([Reaction1, Reaction2, Reaction3, Reaction4])

        self.add_initial_condition(spatialpy.UniformInitialCondition(A, 200, [1]))
        self.add_initial_condition(spatialpy.UniformInitialCondition(B, 200, [1]))
        self.add_initial_condition(spatialpy.UniformInitialCondition(D, 20, [1]))
        self.add_initial_condition(spatialpy.UniformInitialCondition(E, 20, [1]))
        self.add_initial_condition(spatialpy.UniformInitialCondition(X, 20, [1]))
        self.add_initial_condition(spatialpy.UniformInitialCondition(Y, 20, [1]))

        self.staticDomain = True

        self.timespan(range(0,2))

In [30]:
model = turingPattern()

In [31]:
from spatialpy import Solver
sol = Solver(model, debug_level=0)
sol.compile()
%time result = sol.run()

Terminated by user after seconds: 364.09
CPU times: user 8.4 ms, sys: 4.09 ms, total: 12.5 ms
Wall time: 6min 4s


In [17]:
for i,t in enumerate(result.get_timespan()):
    #print(i,t)
    result.plot_species("X", i, use_matplotlib=True)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/spatialpy_result_g8gnh9f7/output100000.vtk'